In [1]:
import os
import numpy as np
from tqdm.notebook import tqdm
import xml.etree.ElementTree as ET
import cv2
import random

from xmlMaker import init_xml
from bbox import bbox_resize
from xmlMaker import add_object
from xmlMaker import prettyXml
from bbox import caculate_area
from bbox import filter_overlapping_boxes

# make XML file

load bounding boxes in txt to generate xml file

In [3]:
xmlFolderPath = os.path.join('../', 'Annotations/')
if not os.path.exists(xmlFolderPath):
    os.mkdir(xmlFolderPath)

widthRatio = 640.0 / 3840.0
heightRatio = 480.0 / 2160.0
# before resize
areaThreshold = 32 / widthRatio / heightRatio

labelsPath = '../labels/'
labelList = os.listdir(labelsPath)

for image in tqdm(labelList):
    imageName = image.split('.')[0]
    xmlPath = os.path.join(xmlFolderPath, '{}.xml'.format(imageName))
    txtPath = os.path.join(labelsPath, image)
    
    with open(txtPath, 'r') as f:
        txtRead = [x.strip()[1:-1]for x in f.readlines()]
    
    bboxes = [x.split() for x in txtRead]
    bboxes = np.array(bboxes).astype(np.int64)
            
    # init xml
    if not os.path.exists(xmlPath):
        init_xml(xmlPath, 'flame', imageName, 'flame_fire_detection', 
                imageWidth=640, imageHeight=480)

    # delete small area bboxes
    area = caculate_area(bboxes)
    mask = area > areaThreshold
    tmpeBboxes = bboxes[mask]
    # delete repetitive bboxes
    bboxesSelected = filter_overlapping_boxes(tmpeBboxes)
    
    for bbox in bboxesSelected:
        bboxResized = bbox_resize(bbox, widthRatio=widthRatio, heightRatio=heightRatio)
        # parse xml file
        tree = ET.parse(xmlPath)
        add_object(tree, bboxResized, 'fire')
        # reshape xml
        root = tree.getroot()
        prettyXml(root, '\t', '\n')
        # save
        tree.write(xmlPath)

  0%|          | 0/2003 [00:00<?, ?it/s]

# generate ImageSets and JPEGImages floders

## make JPEGImages

copy images after reshaping into JPEGImages foldes

In [5]:
jpegimagesPath = '../JPEGImages/'
# make JPEGImages folder
if not os.path.exists(jpegimagesPath):
    os.mkdir(jpegimagesPath)

labelsPath = '../labels/'
labelList = os.listdir(labelsPath)

for image in tqdm(labelList):
    imageName = image.split('.')[0]
    imagePath = '../images/{}.jpg'.format(imageName)
    savePath = os.path.join(jpegimagesPath, '{}.jpg'.format(imageName))
    if not os.path.exists(savePath):
        imageArray = cv2.imread(imagePath)
        imageResized = cv2.resize(imageArray, (640, 480))
        cv2.imwrite(savePath, imageResized)

  0%|          | 0/2003 [00:00<?, ?it/s]

## make ImageSets

loading all image names, distribute them into trainset(70%) and testset(30%)

In [10]:
labelsPath = '../labels/'

allImage = [x.split('.')[0] for x in os.listdir(labelsPath)]

imageSetsPath = '../ImageSets/Main/'
# make ImageSetsPath folder
if not os.path.exists(imageSetsPath):
    os.makedirs(imageSetsPath)
    
allImage = np.array(allImage)

testSize = int(len(allImage) * 0.3)
# shuffle
np.random.shuffle(allImage)
# sample
testIndex = random.sample(range(len(allImage)), testSize)
testList = allImage[np.array(testIndex)]
trainList = [item for item in allImage if item not in testList]
# save
trainTxtPath = os.path.join(imageSetsPath, 'trainval.txt')
testTxtPath = os.path.join(imageSetsPath, 'test.txt')

with open(trainTxtPath, 'w') as f:
    for image in trainList:
        f.write('{}\n'.format(image))

with open(testTxtPath, 'w') as f:
    for image in testList:
        f.write('{}\n'.format(image))